# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: <br>
**qLoRA** (quantized Low Rank Adaptation). It quantized the base model parameters to allow large model to fit in smaller memory. While compute time is increased for quantization and de-quantization, fitting a large model in small memory makes is memory efficient. LoRA on other hand decomposes a large matrix into small matrices, reducing number of parameters. It requires less memory and speeds up fine-tuning.
<br>https://huggingface.co/docs/peft/developer_guides/lora
<br>https://huggingface.co/docs/peft/main/en/developer_guides/quantization
<br>

* Model: <br>
**google-bert/bert-large-uncased** :  
<br>https://huggingface.co/google-bert/bert-large-uncased
<br>

* Evaluation approach: <br>
**seqeval** framework for sequence labeling evaluation. It evaluates the precision, recall and f1 score.
<br>https://huggingface.co/spaces/evaluate-metric/seqeval
<br>

* Fine-tuning dataset: <br>
**financial_phrasebank** based on the financial news, a multi-class-classification with three sentiments (positive, negative and neutral)
<br>https://huggingface.co/datasets/financial_phrasebank


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
!pip install transformers==4.38.2 # --upgrade
!pip install evaluate seqeval
!pip install peft
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 21.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.1
    Uninstalling transformers-4.39.1:
      Successfully uninstalled transformers-4.39.1
  Using cached peft-0.10.0-py3-none-any.whl (199 kB)


#### IMPORTS

In [4]:
from transformers import (AutoModelForSequenceClassification,
                          AutoTokenizer, DataCollatorWithPadding,
                          TrainingArguments, Trainer,
                          BitsAndBytesConfig)

from datasets     import load_dataset

from peft import (LoraConfig, get_peft_model, TaskType,
                  LoftQConfig, prepare_model_for_kbit_training)


import torch
import evaluate
import numpy as np
import pandas as pd


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Define Variables & Load dataset

In [6]:
# https://www.evidentlyai.com/classification-metrics/multi-class-metrics

accuracy  = evaluate.load('accuracy')
f1        = evaluate.load('f1')
precision = evaluate.load('precision')
recall    = evaluate.load('recall')

In [7]:
# checkpoint = "distilbert/distilroberta-base"
checkpoint = 'google-bert/bert-large-uncased'
data_file = "financial_phrasebank"
data_file_subset = "sentences_66agree"

In [8]:
# import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    return accuracy.compute(predictions=predictions, references=labels)

## Load Datasets

In [9]:
def loadDataset(dataFile=data_file, dataFileSubset=data_file_subset, seed=42):
  raw_dataset = load_dataset(path=data_file,
                            name=data_file_subset,
                            split="train").train_test_split(test_size=0.2,
                                                            shuffle=True,
                                                            seed=42)

  raw_train = raw_dataset.pop('train')
  raw_train_valid = raw_train.train_test_split(test_size=.1, shuffle=True, seed=42)
  raw_dataset['train'] = raw_train_valid.pop('train')
  raw_dataset['eval'] = raw_train_valid.pop('test')

  labels = raw_dataset["train"].features['label'].names
  label2id = {l:i for i, l in enumerate(labels)}
  id2label = {i:l for i, l in enumerate(labels)}

  return raw_dataset, label2id, id2label


# Load Model

In [10]:
# Quantize the model

def loadModel(checkpoint=checkpoint, label2id={}, id2label={}):

  quant_config=BitsAndBytesConfig(load_in_4bit = True,
                                  bnb_4bit_quant_type="nf4",
                                  bnb_4bit_use_double_quant=True,
                                  bnb_4bit_compute_dtype=torch.bfloat16
                                  )

  model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                            num_labels = len(label2id),
                                                            id2label=id2label,
                                                            label2id=label2id,
                                                            quantization_config=quant_config,
                                                            device_map={"":0}
                                                            )

  return prepare_model_for_kbit_training(model)


In [11]:
def loadTokenizer(checkpoint=checkpoint):
  return AutoTokenizer.from_pretrained(checkpoint)




In [12]:

def tokenize_function(data, tokenizer):
    return tokenizer(data['sentence'],
#                      max_length=input_max_length,
                     truncation=True,
#                      padding='max_length'
                    )


def get_tokenized_dataset(raw_dataset, tokenizer):
  return raw_dataset.map(tokenize_function, fn_kwargs={"tokenizer": tokenizer}, batched=True)

In [13]:
def get_data_collator(tokenizer):
 return DataCollatorWithPadding(tokenizer=tokenizer,
                                        padding=True,
#                                         padding='max_length',
#                                         max_length=input_max_length)
                                       )


In [14]:
raw_dataset, label2id, id2label = loadDataset(data_file, data_file_subset)
tokenizer = loadTokenizer(checkpoint)
tokenized_dataset = get_tokenized_dataset(raw_dataset, tokenizer)
model= loadModel(checkpoint, label2id, id2label)
data_collator = get_data_collator(tokenizer)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4217 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Map:   0%|          | 0/844 [00:00<?, ? examples/s]

Map:   0%|          | 0/3035 [00:00<?, ? examples/s]

Map:   0%|          | 0/338 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
print("model :\n",model)
print("tokenizer : \n", tokenizer)
print("tokenized_dataset :\n", tokenized_dataset)
print("data_collator : \n", data_collator)

model :
 BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (key): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (value): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (Lay

In [16]:
[len(x) for x in tokenized_dataset['train'][:10]['sentence']]

[120, 75, 152, 46, 152, 90, 50, 167, 160, 49]

In [17]:
[len(x) for x in tokenized_dataset['train'][:10]['input_ids']]

[22, 19, 38, 12, 35, 20, 14, 57, 42, 16]

In [18]:

def evaluate_samples(model=model, ds=tokenized_dataset['train'], sample_start=0, sample_count=10):
  samples = ds[sample_start : sample_start+sample_count]

  samples = {k: v for k, v in samples.items() if k not in ['sentence', 'label']}

  batch = data_collator(samples ).to(device)

  output = model(**batch).logits

  predictions=torch.argmax(output, dim=1).cpu().numpy()

  return predictions


In [19]:
sample_start = 20
sample_count = 10
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


predictions=evaluate_samples(model, tokenized_dataset['train'], sample_start, sample_count)
references =tokenized_dataset['train']['label'][sample_start:sample_start+sample_count]
print("predictions = {}".format(predictions.tolist()))
print("references  = {}".format(references))

predictions = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
references  = [1, 2, 2, 1, 2, 0, 1, 1, 1, 2]


In [20]:


clf =  evaluate.combine(["accuracy",'f1','precision','recall'])
accuracy_metric = accuracy.compute (predictions = predictions, references  = references )
f1_metric       = f1.compute       (predictions = predictions, references  = references,  average = "macro")
precision_metric= precision.compute(predictions = predictions, references  = references,   average = "macro", zero_division=0)
recall_metric   = recall.compute   (predictions = predictions, references  = references,  average = "macro")


print(accuracy_metric)
print(f1_metric)
print(precision_metric)
print(recall_metric)

{'accuracy': 0.1}
{'f1': 0.06060606060606061}
{'precision': 0.03333333333333333}
{'recall': 0.3333333333333333}


In [46]:

def evaluate_model(model=model, sample_start =0, sample_count=4, tokenized_dataset=tokenized_dataset['test']):

    inferences = []
    references = []
    for start in range(sample_start, 844,  sample_count):
      inferences += evaluate_samples(model, tokenized_dataset, start, sample_count).tolist()
      references += tokenized_dataset['label'][start:start+sample_count]


    mismatches = []
    for n, (i, r) in enumerate(zip(inferences, references)):
      if i!=r:
        # mismatches.append(n)
        txt = "prediction: {}, reference: {}, sentence:{}".format(id2label[i], id2label[r], raw_dataset['test'][n]['sentence'])
        mismatches.append(txt)


    return mismatches, (len(mismatches)/len(inferences)), (len(inferences)-len(mismatches))/len(inferences)

In [24]:
mismatches, mismatch_percent, match_percent = evaluate_model(model, 0, 4, tokenized_dataset['test'])
print(len(mismatches), mismatch_percent)

633 0.75


In [25]:
print('mismatch count = ', len(mismatches), 'mismatch percent = ', mismatch_percent, 'match percent = ', match_percent)

mismatch count =  633 mismatch percent =  0.75


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [26]:
tokenized_dataset['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'neutral', 'positive'], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [27]:
tokenized_dataset['train'].rename_column('label','labels')

Dataset({
    features: ['sentence', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3035
})

In [28]:
# config = LoraConfig()

def get_qlora_model():
  loftq_config = LoftQConfig(loftq_bits=4)

  qlora_config = LoraConfig(task_type=TaskType.SEQ_CLS ,
                          inference_mode    = False,
                          #  init_lora_weights = "loftq",
                          #  loftq_config      = loftq_config,
                          r                 = 16,
                          lora_alpha        = 32,
                          lora_dropout      = 0.05,
                          bias              = 'none',
                          target_modules    = ['query','value', 'key',"all-linear"], # 'out_proj'],
                          modules_to_save   = ['classifier']
                          )
  qlora_model = get_peft_model(model, qlora_config )

  return qlora_model


In [29]:
qlora_model = get_qlora_model()
qlora_model.print_trainable_parameters()

trainable params: 2,362,371 || all params: 337,507,334 || trainable%: 0.6999465676796226


In [30]:
qlora_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 1024, padding_idx=0)
          (position_embeddings): Embedding(512, 1024)
          (token_type_embeddings): Embedding(2, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-23): 24 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                   

In [34]:
Training_Arguments = TrainingArguments(
    per_device_train_batch_size = 4,
    per_device_eval_batch_size  = 4,
    output_dir                  = "bert_large_qlora_classifier",
    learning_rate               = 2e-5,
    num_train_epochs            = 15,
    weight_decay                = 0.005,
    save_strategy               = 'epoch',
    evaluation_strategy         = 'epoch',
    deepspeed                   = False,
    load_best_model_at_end      = True)

In [35]:
trainer = Trainer(
                  model=qlora_model,
                  args=Training_Arguments,
                  train_dataset=tokenized_dataset['train'],
                  eval_dataset =tokenized_dataset['eval'],
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer,
                  data_collator=data_collator
)

In [36]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.938500,0.723656,0.677515
2,0.539000,0.465848,0.828402
3,0.456400,0.421799,0.866864
4,0.412700,0.449275,0.881657
5,0.396600,0.383018,0.890533
6,0.374200,0.398774,0.896450
7,0.400900,0.407567,0.896450
8,0.367000,0.394693,0.899408
9,0.306500,0.366017,0.905325
10,0.323900,0.363788,0.914201


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

TrainOutput(global_step=11385, training_loss=0.395496618030466, metrics={'train_runtime': 3326.3415, 'train_samples_per_second': 13.686, 'train_steps_per_second': 3.423, 'total_flos': 3818722332505500.0, 'train_loss': 0.395496618030466, 'epoch': 15.0})

In [37]:
saved_checkpoint = '/content/drive/MyDrive/ftMLC-large-bert-uncased-QLora-Mar-26-00'

In [38]:
trainer.save_model(saved_checkpoint)

In [39]:
print(trainer.evaluate())

{'eval_loss': 0.34605905413627625, 'eval_accuracy': 0.9260355029585798, 'eval_runtime': 6.8472, 'eval_samples_per_second': 49.363, 'eval_steps_per_second': 12.414, 'epoch': 15.0}


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [40]:
trainer2 = Trainer(
                  model=qlora_model,
                  args=Training_Arguments,
                  train_dataset=tokenized_dataset['train'],
                  eval_dataset =tokenized_dataset['test'],
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer,
                  data_collator=data_collator
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [41]:
print(trainer2.evaluate())

{'eval_loss': 0.5063890814781189, 'eval_accuracy': 0.9004739336492891, 'eval_runtime': 17.5729, 'eval_samples_per_second': 48.028, 'eval_steps_per_second': 12.007}


In [47]:
mismatches, mismatch_percent, match_percent = evaluate_model(qlora_model, 0, 4, tokenized_dataset['test'])
print("post training")
# print('mismatch count = ', len(mismatches), 'mismatch percent = ', mismatch_percent*100)
print('mismatch count = ', len(mismatches), 'mismatch %age = ', mismatch_percent*100, 'match %age = ', match_percent*100)

post training
mismatch count =  84 mismatch %age =  9.95260663507109 match %age =  90.04739336492891


In [ ]:
# print(evaluate_samples(qlora_model, tokenized_dataset['test'], 10, 50))

[2 2 1 1 0 1 1 1 1 0 1 1 2 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 1 2 2 1 1 1 2 1 1
 1 2 1 2 1 0 0 2 1 0 1 1 1]


In [48]:

# sample_start=20
# sample_count=50

# inferences = evaluate_samples(qlora_model, tokenized_dataset['test'], sample_start, sample_count).tolist()
# print(inferences)

# references=tokenized_dataset['test']['label'][sample_start:sample_start+sample_count]
# print(references)

In [ ]:

# sample_start=0
# sample_count=4

# inferences = []
# references = []
# for i in range(sample_start, 844,  sample_count):
#   inferences += evaluate_samples(qlora_model, tokenized_dataset['test'], sample_start, sample_count).tolist()
#   references += tokenized_dataset['test']['label'][sample_start:sample_start+sample_count]



In [ ]:
# mismatches = []
# for n, (i, r) in enumerate(zip(inferences, references)):
#   if i!=r:
#     # mismatches.append(n)
#     txt = "prediction: {}, reference: {}, sentence:{}".format(id2label[i], id2label[r], raw_dataset['test'][n]['sentence'])
#     mismatches.append(txt)

In [ ]:
# print(len(mismatches))

422


In [ ]:
# print(len(mismatches)/len(inferences))

0.5


## Accuracy on the test set is 50% after training for 5 epochs. Training for additional 10 epochs

In [ ]:
Training_Arguments = TrainingArguments(
    per_device_train_batch_size = 4,
    per_device_eval_batch_size  = 4,
    output_dir                  = "bert_large_qlora_classifier2",
    learning_rate               = 2e-5,
    num_train_epochs            = 10,
    weight_decay                = 0.005,
    save_strategy               = 'epoch',
    evaluation_strategy         = 'epoch',
    deepspeed                   = False,
    load_best_model_at_end      = True)
trainer = Trainer(
                  model=qlora_model,
                  args=Training_Arguments,
                  train_dataset=tokenized_dataset['train'],
                  eval_dataset =tokenized_dataset['eval'],
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer,
                  data_collator=data_collator
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.417800,0.420329,0.893491
2,0.424200,0.361170,0.905325
3,0.357200,0.381627,0.902367
4,0.322900,0.397611,0.899408
5,0.299500,0.348891,0.917160
6,0.312100,0.354088,0.920118
7,0.308400,0.322848,0.928994
8,0.273900,0.341263,0.923077
9,0.275000,0.324152,0.931953
10,0.268300,0.330553,0.926036


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

TrainOutput(global_step=7590, training_loss=0.32756006092579154, metrics={'train_runtime': 2250.7859, 'train_samples_per_second': 13.484, 'train_steps_per_second': 3.372, 'total_flos': 2545257243255720.0, 'train_loss': 0.32756006092579154, 'epoch': 10.0})

In [ ]:
saved_checkpoint = '/content/drive/MyDrive/ftMLC-large-bert-uncase-QLora-Mar-25-01'

In [ ]:
trainer.save_model(saved_checkpoint)

In [ ]:
print(trainer.evaluate())

{'eval_loss': 0.32284772396087646, 'eval_accuracy': 0.9289940828402367, 'eval_runtime': 13.843, 'eval_samples_per_second': 24.417, 'eval_steps_per_second': 6.14, 'epoch': 10.0}


In [ ]:
trainer2 = Trainer(
                  model=qlora_model,
                  args=Training_Arguments,
                  train_dataset=tokenized_dataset['train'],
                  eval_dataset =tokenized_dataset['test'],
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer,
                  data_collator=data_collator
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:

sample_start=20
sample_count=50

inferences = evaluate_samples(qlora_model, tokenized_dataset['test'], sample_start, sample_count).tolist()
print(inferences)

references=tokenized_dataset['test']['label'][sample_start:sample_start+sample_count]
print(references)

[1, 1, 2, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 0]
[2, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 0]


In [ ]:

sample_start=0
sample_count=4

inferences = []
references = []
for i in range(sample_start, 844,  sample_count):
  inferences += evaluate_samples(qlora_model, tokenized_dataset['test'], sample_start, sample_count).tolist()
  references += tokenized_dataset['test']['label'][sample_start:sample_start+sample_count]



In [ ]:
mismatches = []
for n, (i, r) in enumerate(zip(inferences, references)):
  if i!=r:
    # mismatches.append(n)
    txt = "prediction: {}, reference: {}, sentence:{}".format(id2label[i], id2label[r], raw_dataset['test'][n]['sentence'])
    mismatches.append(txt)

In [ ]:
print(len(mismatches))


0


In [ ]:
print(len(mismatches)/len(inferences))

0.0


In [ ]:
mismatches

[]

## Test set prediction is at 100% after 15 epochs

# Load saved model for inference

In [49]:
saved_checkpoint = '/content/drive/MyDrive/ftMLC-large-bert-uncased-QLora-Mar-26-00'

In [50]:
from peft import PeftConfig, PeftModel, AutoPeftModelForSequenceClassification

In [51]:
config = PeftConfig.from_pretrained(saved_checkpoint)
base_model = AutoModelForSequenceClassification.from_pretrained(
    config.base_model_name_or_path,
    label2id=label2id,
    id2label=id2label,
    # ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint

)
# base_model.to(device)

# Load the qLoRA model
base_tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
inference_model = PeftModel.from_pretrained(base_model, saved_checkpoint )
inference_model = inference_model.merge_and_unload()

inference_model.to(device)
# model.save_pretrained("merged_adapters")
# inference_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,

In [53]:
mismatches, mismatch_percent, match_percent = evaluate_model(inference_model, 0, 4, tokenized_dataset['test'])

print("loading and infereing from saved model")

print('mismatch count = ', len(mismatches), 'mismatch %age = ', mismatch_percent*100, 'match %age = ', match_percent*100)

loading and infereing from saved model
mismatch count =  84 mismatch %age =  9.95260663507109 match %age =  90.04739336492891


In [ ]:

sample_start=20
sample_count=50

inferences = evaluate_samples(inference_model, tokenized_dataset['test'], sample_start, sample_count).tolist()
print(inferences)

references=tokenized_dataset['test']['label'][sample_start:sample_start+sample_count]
print(references)

[1, 1, 2, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 2, 0, 1, 1, 0]
[2, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 0]


In [ ]:

sample_start=0
sample_count=4

inferences = []
references = []
for i in range(sample_start, 844,  sample_count):
  inferences += evaluate_samples(inference_model, tokenized_dataset['test'], sample_start, sample_count).tolist()
  references += tokenized_dataset['test']['label'][sample_start:sample_start+sample_count]



In [ ]:
infer_mismatches = []
for n, (i, r) in enumerate(zip(inferences, references)):
  if i!=r:
    txt = "prediction: {}, reference: {}, sentence:{}".format(id2label[i], id2label[r], raw_dataset['test'][n]['sentence'])
    infer_mismatches.append(txt)

In [ ]:
len(infer_mismatches)

211

In [ ]:
inference_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,

In [ ]:
base_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,

In [ ]:
inference_model.num_parameters()

335144963